LinkedIn jobs web scraper API using Python, Beautiful Soup and Flask with SQLite database

Implementation of LinkedinJobsScraper class to extract the data

In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import pandas as pd
import re
import asyncio
import aiohttp

class LinkedinJobScraper:

    def __init__(self, keywords: str="Software Developer.Engineer", location: str="Dublin, Ireland" ):
        self.base_url='https://www.linkedin.com'
        self.request_url='/jobs/search'
        self.more_request_url='/jobs-guest/jobs/api/seeMoreJobPostings/search'
        self.keywords=keywords
        self.location=location
        self.params = {
            #"geoId": "105178154",
            "trk": "public_jobs_jobs-search-bar_search-submit",
            "keywords": self.keywords,
            "location":self.location
        }

        CURRENCY_SYMBOL = r'(?:[\$€£₹]|Rs\.?)'
        K_SUFFIX = r'[kK]'
        NUMBER_FORMAT = r'\d{1,3}[,.\d]{0,6}'
        SALARY_UNIT_WITH_CURRENCY = f'{CURRENCY_SYMBOL}\\s*{NUMBER_FORMAT}\\s*{K_SUFFIX}?'
        SALARY_UNIT_WITH_OPTIONALCURRENCY = f'{CURRENCY_SYMBOL}?\\s*{NUMBER_FORMAT}\\s*{K_SUFFIX}?'

        self.salary_pattern = f'{SALARY_UNIT_WITH_CURRENCY}(?:\\s*[–-]\\s*{SALARY_UNIT_WITH_OPTIONALCURRENCY})?'
        self.headers={
          "authority": "www.linkedin.com",
          "method": "GET",
          "path": f"/jobs/search?keywords=Software%20Developer&location=Dublin%2C%20County%20Dublin%2C%20Ireland&geoId=105178154&trk=public_jobs_jobs-search-bar_search-submit",
          "scheme": "https",
          "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
          "accept-encoding": "gzip, deflate, br, zstd",
          "accept-language": "en-US,en;q=0.9",
          "cache-control": "max-age=0",
          "cookie": "JSESSIONID=ajax:4227796472434740136; lang=v=2&lang=en-us; bcookie=\"v=2&b2e99f24-0f23-42d4-8d7a-9ce9eb7d2fb2\"; bscookie=\"v=1&20251102233949c5da3359-f509-418c-885d-b95c4d306fc7AQHTQYDPTg81tF76fv8ifXyoI2kwnWvP\"; lidc=\"b=VGST07:s=V:r=V:a=V:p=V:g=3380:u=1:x=1:i=1762126790:t=1762213190:v=2:sig=AQGMPyjM1ixjmdbMdhw9aoW9zXelMXwW\"; li_alerts=e30=; li_gc=MTsyMTsxNzYyMTI2NzkyOzI7MDIxWa4i8DG++7+PUhn+RLXBDYVZjSDbhY/0koOaUifNJn4=; AMCVS_14215E3D5995C57C0A495C55%40AdobeOrg=1; AMCV_14215E3D5995C57C0A495C55%40AdobeOrg=-637568504%7CMCIDTS%7C20395%7CMCMID%7C48232966877475470444279466292121689293%7CMCAAMLH-1762731594%7C6%7CMCAAMB-1762731594%7C6G1ynYcLPuiQxYZrsz_pkqfLG9yMXBpb2zX5dvJdYQJzPXImdj0y%7CMCOPTOUT-1762133994s%7CNONE%7CvVersion%7C5.1.1; aam_uuid=47650612458427710554299979797558800134; _gcl_au=1.1.1125007778.1762126826; _uetsid=4ebcb140b84511f09378372580470222; _uetvid=4ebcbc60b84511f0b0a60d4fa1a52ffa; __cf_bm=p87vKKOxUMaewGjC_Fps7F1YeXpKNVuUvwTW6yI3Evg-1762126852-1.0.1.1-fTdSHgW2BmOd_jOzvp1x_LZfZdW5rX4Pgzv3hXs6CT1gEk0xkIWgyNs1IPkMJFfdfkkQpKcE909ZmJiDYMOO7j7O6AuD62SnrdM.ozoIjtE",
          "priority": "u=0, i",
          "referer": "https://www.linkedin.com/jobs/search?keywords=Software%20developer&location=Ireland&geoId=105178154&trk=public_jobs_jobs-search-bar_search-submit",
          "sec-ch-ua": "\"Microsoft Edge\";v=\"141\", \"Not?A_Brand\";v=\"8\", \"Chromium\";v=\"141\"",
          "sec-ch-ua-mobile": "?1",
          "sec-ch-ua-platform": "\"Android\"",
          "sec-fetch-dest": "document",
          "sec-fetch-mode": "navigate",
          "sec-fetch-site": "same-origin",
          "sec-fetch-user": "?1",
          "upgrade-insecure-requests": "1",
          "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Mobile Safari/537.36 Edg/141.0.0.0"
        }

        if 'cookie' in self.headers: del self.headers['cookie']
        if 'path' in self.headers: del self.headers['path']
        if 'referer' in self.headers: del self.headers['referer']

    def get_job_cards(self, startNumber,current_req_type='initial'):
        if current_req_type == 'initial':
          current_url=self.request_url
        else:
          current_url=self.more_request_url

        self.headers['referer']=f"{self.base_url+current_url}?{urllib.parse.urlencode(self.params)}"
        if startNumber!=0:
          self.params['start']=startNumber

        curr_request_url = f"{self.base_url+current_url}?{urllib.parse.urlencode(self.params)}"
        self.headers['path']=f"{current_url}?{urllib.parse.urlencode(self.params)}"

        response = requests.get(curr_request_url,headers=self.headers)
        soup = BeautifulSoup(response.content,'lxml')
        #print(response.content)
        job_cards = soup.find_all('div', class_='base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card')

        return job_cards

    def get_job_details(self, href_link):
        try:
            response = requests.get(href_link)
            soup = BeautifulSoup(response.content,'lxml')
            job_criteria_ul = soup.find('ul', class_='description__job-criteria-list')
            job_description_div = soup.find('div', class_='show-more-less-html__markup')

            #creating new soup obj to store the details
            container = BeautifulSoup('<div></div>', 'lxml').div
            if job_criteria_ul:
              container.append(job_criteria_ul)
            if job_description_div:
              container.append(job_description_div)

            return container
        except Exception as e:
            print(f"Error fetching job details: {e}")
            return None

    async def get_job_details_async(self, session: aiohttp.ClientSession, href_link: str):

        if not href_link:
            return None

        try:
            async with session.get(href_link) as resp:
                content = await resp.read()
                soup = BeautifulSoup(content,'lxml')
                job_criteria_ul = soup.find('ul', class_='description__job-criteria-list')
                job_description_div = soup.find('div', class_='show-more-less-html__markup')

                #creating new soup obj to store the details
                container = BeautifulSoup('<div></div>', 'lxml').div
                if job_criteria_ul:
                  container.append(job_criteria_ul)
                if job_description_div:
                  container.append(job_description_div)

                return container
        except Exception as e:
            print(f"Error fetching job details: {e}")
            return None

    def extract_salary_from_jobdescription(self, job_description):
        soup = BeautifulSoup(job_description, "html.parser")
        plain_text = soup.get_text(separator=" ")
        matches = re.findall(self.salary_pattern, plain_text)

        valid_matches=[]
        for matche in matches:
            if re.search(r'\b\d+\.?\d*\s*[Mm]\b', matche):
                continue
            if re.search(r'\bmillion\b', matche, re.IGNORECASE):
                continue

            valid_matches.append(matche)

        for matche in valid_matches:
            if '–' in matche or '-' in matche:
                return matche

        if valid_matches:
            return valid_matches[0]

        return "N/A"

    def get_jobs_list(self, cards, use_async:bool=True, max_workers:int=8,time_out:int=10):
        job_list = []
        temp_job_list=[]
        for card in cards:
            #Look for the elements containing job title, company, location, and posted date
            job_link_elem= card.find('a', class_='base-card__full-link')
            title_elem = card.find('h3', class_='base-search-card__title')
            company_elem = card.find('h4', class_='base-search-card__subtitle')
            location_elem = card.find('span', class_='job-search-card__location')
            date_elem = card.find('time', class_='job-search-card__listdate')

            job_link=job_link_elem['href'] if job_link_elem else 'N/A'
            title = title_elem.text.strip() if title_elem else 'N/A'
            company = company_elem.text.strip() if company_elem else 'N/A'
            location = location_elem.text.strip() if location_elem else 'N/A'
            date = date_elem.text.strip() if date_elem else 'N/A'

            temp_job_list.append({
                'Job Title': title,
                'Job_Link': job_link,
                'Company': company,
                'Location': location,
                'Posted Date': date
              })

        #adding request processing for every job detail asynchronously
        links =[link['Job_Link'] for link in temp_job_list]
        details_results = [None]*len(links)

        if use_async and any(links):
            aio_headers={
                'User-Agent': self.headers.get('user-agent','python-requests/0'),
                'Referer': self.headers.get('referer',''),
            }

            #inner function to create tasks to extract job details
            async def _gather():
                sem = asyncio.Semaphore(max_workers)
                async def _fetch(idx,link):
                    async with sem:
                        async with aiohttp.ClientSession(headers=aio_headers) as session:
                            try:
                                res=await self.get_job_details_async(session,link)
                                return idx,res
                            except Exception:
                                return idx,None

                tasks=[_fetch(idx,link) for idx,link in enumerate(links)]
                completed=await asyncio.gather(*tasks)
                return completed

            try:
                completed_results=asyncio.run(_gather())
                for idx,res in completed_results:
                    details_results[idx]=res
            except Exception:
                for idx,link in enumerate(links):
                    details_results[idx]=self.get_job_details(link) if link else None
        else:
            for idx,link in enumerate(links):
                details_results[idx]=self.get_job_details(link) if link else None

        for idx,meta in enumerate(temp_job_list):
            job_details=details_results[idx]
            job_details_elem = job_details.find_all('li') if job_details else []
            seniority_level = 'N/A'
            employment_type = 'N/A'
            job_function = 'N/A'
            industries = 'N/A'

            #assigning the four values lists to respective attribute
            if len(job_details_elem) > 0:
                seniority_level_span = job_details_elem[0].find('span')
                seniority_level = seniority_level_span.text.strip() if seniority_level_span else 'N/A'
            if len(job_details_elem) > 1:
                employment_type_span = job_details_elem[1].find('span')
                employment_type = employment_type_span.text.strip() if employment_type_span else 'N/A'
            if len(job_details_elem) > 2:
                job_function_span = job_details_elem[2].find('span')
                job_function = job_function_span.text.strip() if job_function_span else 'N/A'
            if len(job_details_elem) > 3:
                industries_span = job_details_elem[3].find('span')
                industries = industries_span.text.strip() if industries_span else 'N/A'

            #extract and setting salary by applying regex pattern from job description
            job_description_elem = job_details.find('div', class_='show-more-less-html__markup')
            job_description = job_description_elem.text.strip() if job_description_elem else 'N/A'
            salary=self.extract_salary_from_jobdescription(job_description)
            #job_description = BeautifulSoup(job_description, 'html.parser').get_text(separator=' ')

            job_list.append({
                'Job Title': meta['Job Title'],
                'Job_Link': meta['Job_Link'],
                'Company': meta['Company'],
                'Seniority Level': seniority_level,
                'Employment Type': employment_type,
                'Salary': salary,
                'Job Function': job_function,
                'Industries': industries,
                'Job Description': job_description,
                'Location': meta['Location'],
                'Posted Date': meta['Posted Date']
            })

        return job_list

    def scrape_jobs(self, max_start=200, keywords: str=None,location:str=None,use_async:bool=True,max_workers:int=8):
        totalJobs = []
        start = 0
        curr_req_type='initial'

        if keywords:
          self.keywords = keywords
          self.params['keywords']=keywords
        if location:
          self.location=location
          self.params['location']=location

        while start <= max_start:
            curr_job_cards = self.get_job_cards(start,curr_req_type)
            start += 25
            curr_req_type='more'
            curr_job_list = self.get_jobs_list(curr_job_cards,use_async,max_workers)
            curr_jobs_df = pd.DataFrame(curr_job_list)
            totalJobs.append(curr_jobs_df)

        final_jobs_df = pd.concat(totalJobs, ignore_index=True)
        return final_jobs_df


Implementation of Feature extraction to extract new features from existing ones

In [ ]:
from typing import List,Dict
import re

class FeatureExtraction:

    #programming languages
    PROGRAMMING_LANGUAGES={
    'python', 'java', 'javascript', 'typescript', 'c++', 'c#', 'ruby',
        'php', 'go', 'rust', 'swift', 'kotlin', 'scala', 'r', 'matlab',
        'perl', 'bash', 'shell', 'sql', 'html', 'css', 'react', 'angular',
        'vue', 'node', 'django', 'flask', 'spring', '.net'
    }

    TECHNOLOGIES = {
      'docker', 'kubernetes', 'aws', 'azure', 'gcp', 'terraform', 'jenkins',
        'git', 'linux', 'mysql', 'postgresql', 'mongodb', 'redis', 'elasticsearch',
        'kafka', 'rabbitmq', 'rest', 'graphql', 'microservices', 'api', 'ci/cd',
        'agile', 'scrum', 'tensorflow', 'pytorch', 'machine learning', 'ai',
        'data science', 'big data', 'hadoop', 'spark'
    }

    EDUCATION_KEYWORDS={
      'bachelor', 'master', 'phd', 'degree', 'bs', 'ms', 'ba', 'ma',
        'computer science', 'engineering', 'stem'
    }


    def extract_features_in_a_job(self, job_data: Dict) -> Dict:
        features={
            'title': job_data.get('Job Title',''),
            'job_link': job_data.get('Job_Link',''),
            'company': job_data.get('Company',''),
            'seniority_level': job_data.get('Seniority Level',''),
            'employment_type': job_data.get('Employment Type',''),
            'salary': job_data.get('Salary',''),
            'job_function': job_data.get('Job Function',''),
            'industries': job_data.get('Industries',''),
            'job_description': job_data.get('Job Description',''),
            'location': job_data.get('Location',''),
            'posted_date': job_data.get('Posted Date','')
        }

        #combine necessary column values to extract features
        combined_data=self.combine_fields_for_text(job_data)

        #extract feature from each or combined columns to create new features
        features.update({
            'programming_languages': self.extract_programming_languages(combined_data),
            'technologies': self.extract_technologies(combined_data),
            'education_requirements': self.extract_education_requirements(combined_data),
            'key_skills': self.extract_key_skills(combined_data),
            'salary_range': self.extract_salary_range(job_data)
        })

        return features


    def combine_fields_for_text(self, job_data: Dict):
      combined_text = ''

      text_feilds=[
        job_data.get('Job Title',''),
        job_data.get('Job Description',''),
        job_data.get('Company',''),
      ]
      combined_text += ' '.join(text_feilds)
      return combined_text


    #extract programming languages
    def extract_programming_languages(self, job_description: str):
        programming_languages=[]
        text_lower=job_description.lower()

        for prog_lang in self.PROGRAMMING_LANGUAGES:
            prog_pattern=r'\b' + re.escape(prog_lang) + r'\b'
            if re.search(prog_pattern,text_lower):
              programming_languages.append(prog_lang.title())

        return programming_languages

    #extract technolgies
    def extract_technologies(self, job_description: str):
        technologies=[]
        text_lower=job_description.lower()

        for tech in self.TECHNOLOGIES:
          tech_pattern=r'\b' + re.escape(tech) + r'\b'
          if re.search(tech_pattern,text_lower):
            technologies.append(tech.title())

        return technologies

    #extract education requirements
    def extract_education_requirements(self, job_description: str):
        education_requirements=[]
        text_lower=job_description.lower()

        for edu_req in self.EDUCATION_KEYWORDS:
          edu_pattern=r'\b' + re.escape(edu_req) + r'\b'
          if re.search(edu_pattern,text_lower):
            education_requirements.append(edu_req.title())

        return education_requirements

    #extract salary range
    def extract_salary_range(self, job_data:Dict):
        salary_range={}
        salary_str=job_data.get('Salary','')
        if not salary_str:
          return None

        #numbers_raw_matches = re.findall(r'(?:₹|\$|€|Rs\.?)?\s*(\d{1,3}(?:[.,]\d{3})*|\d+(?:\.\d+)?)\s*(k|K)?',str(salary_str),flags=re.IGNORECASE)
        numbers_raw_matches = re.findall(r'(?:₹|\$|€|Rs\.?)?\s*(\d+(?:[.,]\d{3})*(?:\.\d+)?)\s*(k|K)?',str(salary_str),flags=re.IGNORECASE)

        updated_numbers=[]

        for val,suffix in numbers_raw_matches:
            extracted_num=val.replace(',','')
            if not extracted_num:
                continue

            try:
                num_val=float(extracted_num)
                if suffix and suffix.upper() == 'K':
                    num_val *= 1000
                updated_numbers.append(int(num_val))
            except ValueError:
                print(f'Invalid numerical value{extracted_num}')
                continue

        if not updated_numbers:
            return None

        if len(updated_numbers) >=2:
            salary_range['min']=min(updated_numbers[0],updated_numbers[1])
            salary_range['max']=max(updated_numbers[0],updated_numbers[1])
            return salary_range
        elif len(updated_numbers)==1:
            salary_range['min']=updated_numbers[0]
            salary_range['max']=updated_numbers[0]
            return salary_range

        return None

    #extract key skills
    def extract_key_skills(self, job_description: str):
        key_skills=[]
        text_lower=job_description.lower()
        for skill in (self.PROGRAMMING_LANGUAGES | self.TECHNOLOGIES):
          pattern = r'\b' + re.escape(skill) + r'\b'
          if re.search(pattern, text_lower):
              key_skills.append(skill.title())

        return key_skills

Implementation of Data Transformer to clean and transform the data into structured and unified format

In [ ]:
from typing import List,Dict
from datetime import datetime

class DataTransformer:
    def transform_all_jobs(self, all_jobs: List[Dict]) -> List[Dict]:
        transformed_jobs = []
        for job in all_jobs:
            try:
                transformed_job = self.transform_job(job)
                transformed_jobs.append(transformed_job)
            except Exception as e:
                print(f"Error transforming job: {job.get('title', 'Unknown')}- {e}")
                continue
        return transformed_jobs

    def transform_job(self, job_data: Dict) -> Dict:
        transformed_job = job_data.copy()

        #clean and normalize each column values
        transformed_job['title'] = job_data.get('title', '')
        transformed_job['company'] = job_data.get('company', '')
        transformed_job['location'] = job_data.get('location', '')
        transformed_job['job_link'] = job_data.get('job_link', '')
        transformed_job['seniority_level'] = job_data.get('seniority_level', '')
        transformed_job['employment_type'] = job_data.get('employment_type', '')
        transformed_job['salary'] = job_data.get('salary', '')
        transformed_job['job_description'] = self._clean_description(job_data.get('job_description', ''))
        transformed_job['posted_date'] = self._normalize_datetime(job_data.get('posted_date', ''))
        transformed_job['programming_languages'] = self._normalize_list(job_data.get('programming_languages', []))
        transformed_job['technologies'] = self._normalize_list(job_data.get('technologies', []))
        transformed_job['key_skills'] = self._normalize_list(job_data.get('key_skills', []))
        transformed_job['salary_max'] = self._extract_salary_max(job_data.get('salary_range'))
        transformed_job['salary_min'] = self._extract_salary_min(job_data.get('salary_range'))

        return transformed_job

    # #clean text
    # def _clean_text(self, text:str):
    #     if not text:
    #         return ''
    #     text = text.lower()
    #     text = re.sub(r'\s+', ' ', text)
    #     return text.strip()

    #clean description
    def _clean_description(self, description:str):
        if not description:
            return ''
        description = re.sub(r'<[^>]+>', '', description)
        description = re.sub(r'\n{3,}', '\n\n', description)
        description = re.sub(r'\s+', ' ', description)

        # if len(description)>5000:
        #     description=description[:5000]+'....'
        return description.strip()

    # #normalize locations
    # def _normalize_location(self, location:str):
    #     if not location:
    #         return 'Not specified'
    #     return self._clean_text(location)


    #normalize datetime
    def _normalize_datetime(self, datetimestr:str):
        if not datetimestr:
            return None

        try:
            date_formats = [
                '%Y-%m-%d',
                '%m/%d/%Y',
                '%d/%m/%Y',
                '%Y-%m-%d %H:%M:%S',
            ]

            for date_format in date_formats:
                try:
                    parsed_date = datetime.strptime(datetimestr, date_format)
                    return parsed_date.isoformat()
                except ValueError:
                    continue

            return None
        except Exception as e:
            print(f"Error normalizing datetime: {e}")
            return None

    #extract salary max
    def _extract_salary_max(self, salary_range: Dict):
        if salary_range and isinstance(salary_range,dict):
            return salary_range.get('max')
        return None

    #extract salary min
    def _extract_salary_min(self, salary_range: Dict):
        if salary_range and isinstance(salary_range,dict):
            return salary_range.get('min')
        return None

    #remove duplicates
    def _remove_duplicates(self, all_jobs: List[Dict]) -> List[Dict]:
        key_fields=['title', 'company', 'location']
        unique_jobs = []
        seen_jobs = set()

        for job in all_jobs:
            key_parts = [str(job.get(field,'')).lower().strip() for field in key_fields]
            key = '|'.join(key_parts)

            if key not in seen_jobs:
                unique_jobs.append(job)
                seen_jobs.add(key)

        return unique_jobs

    #convert list into comma separated string
    def _normalize_list(self, lst: List):
        if not lst:
            return ''

        return ', '.join(sorted(set(lst)))


Implementation if SQLite3 database and table creation. And dumping the data into it.

In [ ]:
import sqlite3
import pandas as pd


class DatabaseManager:
    def __init__(self, db_name='jobs.db'):
        self.db_name=db_name
        self.conn=None
        self.cursor=None


    def _connect(self):
        try:
            self.conn = sqlite3.connect(self.db_name,check_same_thread=False)
            self.cursor = self.conn.cursor()
        except sqlite3.error as e:
            print(f"Error connecting to database: {e}")

    def _close(self):
        if self.conn:
            try:
                self.conn.close()
            except Exception:
                pass
        self.conn=None
        self.cursor=None

    #create table
    def create_table(self, table_name='jobs'):
        self.table_name = table_name
        if self.conn is None:
            self._connect()
        try:
            self.cursor.execute(f'''
                CREATE TABLE IF NOT EXISTS {self.table_name}(
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    title TEXT,
                    job_link TEXT,
                    company TEXT,
                    seniority_level TEXT,
                    employment_type TEXT,
                    salary TEXT,
                    job_function TEXT,
                    industries TEXT,
                    job_description TEXT,
                    location TEXT,
                    posted_date TEXT,
                    programming_languages TEXT,
                    technologies TEXT,
                    key_skills TEXT,
                    salary_max INTEGER,
                    salary_min INTEGER
                )
             ''')
            self.conn.commit()
            print('Table created successfully')
        except sqlite3.Error as e:
            print(f"Error creating table: {e}")

    #insert data into table
    def insert_data(self, df: pd.DataFrame):
        if self.conn is None:
            self._connect()

        df_filtered = df[[col for col in df.columns if col in self._get_table_columns()]]
        df_filtered = df_filtered.where(pd.notnull(df_filtered), None)

        columns = ', '.join(df_filtered.columns)
        placeholders = ', '.join(['?' for _ in df_filtered.columns])
        insert_query = f"INSERT INTO {self.table_name} ({columns}) VALUES ({placeholders})"

        try:
            self.cursor.executemany(insert_query, df_filtered.values.tolist())
            self.conn.commit()
            print(f'Inserted the data into {self.table_name} successfully with {len(df_filtered)} records')
        except sqlite3.Error as e:
            print(f"Error inserting data: {e}")

    #get the data
    def get_data(self):
        try:
            if self.conn is None:
                self._connect()
            self.cursor.execute(f"SELECT * FROM {self.table_name}")
            columns = [desc[0] for desc in self.cursor.description]
            rows = self.cursor.fetchall()
            return pd.DataFrame(rows, columns=columns)
        except sqlite3.Error as e:
            print(f"Error getting data: {e}")
            return pd.DataFrame()

    #refresh the data
    def refresh_data(self, df: pd.DataFrame):
        try:
            if self.conn is None:
                self._connect()
            self.cursor.execute(f"DELETE FROM {self.table_name}")
            self.conn.commit()
            self.insert_data(df)
        except sqlite3.Error as e:
            print(f"Error refreshing data: {e}")

    #reset the database
    def reset_database(self):
        try:
            if self.conn is None:
                self._connect()
            self.cursor.execute(f"DROP TABLE IF EXISTS {self.table_name}")
            self.conn.commit()
            self.create_table()
            print(f"Database reset successfully")
        except sqlite3.Error as e:
            print(f"Error resetting database: {e}")

    #get table columns
    def _get_table_columns(self):
        if self.conn is None:
            self._connect()
        self.cursor.execute(f"PRAGMA table_info({self.table_name})")
        columns = [column[1] for column in self.cursor.fetchall() if column[1]!='id']
        return columns


Implementation of LinkedinJobsPipeline to stremline the method callings for jobs extraction and utilizing this in API endpoints logic

In [ ]:
class LinkedinJobsPipeline:
    def __init__(self, db_name='jobs.db'):
        self.scraper=LinkedinJobScraper()
        self.extractor=FeatureExtraction()
        self.data_transformer=DataTransformer()
        self.db_manager=DatabaseManager(db_name=db_name)

    def run_jobs_extraction_pipeline(self, keywords:str, location:str, limit:int=200, isRefreshData=False,use_async:bool=True,max_workers:int=8):
        #scrap jobs
        scraped_jobs_df=self.scraper.scrape_jobs(limit,keywords,location,use_async=use_async,max_workers=max_workers)
        if scraped_jobs_df.empty:
            print('No data scraped')
            return pd.DataFrame()

        scraped_jobs_list=scraped_jobs_df.to_dict(orient='records')
        print(f'Scraped {len(scraped_jobs_list)} jobs')

        #extract features
        extracted_jobs=[]
        for job in scraped_jobs_list:
            try:
                features=self.extractor.extract_features_in_a_job(job)
                extracted_jobs.append(features)
            except Exception as e:
                print(f"Error extracting features from job: {job.get('title', 'Unknown')}- {e}")
                continue
        print(f'Extracted features for {len(extracted_jobs)} jobs')

        #data transformation
        transformed_jobs=self.data_transformer.transform_all_jobs(extracted_jobs)
        transformed_jobs=self.data_transformer._remove_duplicates(transformed_jobs)
        transformed_jobs_df = pd.DataFrame(transformed_jobs)

        if transformed_jobs_df.empty:
            print('No data to dump after extraction and transformation')
            return pd.DataFrame()

        #data dumping
        self.db_manager.create_table()
        if isRefreshData:
            self.db_manager.refresh_data(transformed_jobs_df)
        else:
          self.db_manager.insert_data(transformed_jobs_df)

        self.db_manager._close()
        return transformed_jobs_df


Sample Pipeline Run

In [ ]:
#keywords
#Role specific - Dotnet Fullstack Developer,Python Developer,Data Scientist,AI Engineer,ML Engineer,Backend Engineer/Developer,Frontend Engineer/Developer
#Company wise - Zoho, Google, Amazon, Tesla, SpaceX, Microsoft, OpenAI, Anthrophic, TCS, Infosys, Cognizant, Wipro, Statestreet, JP Morgan, Vanguards, Pwc
#Software Developer/Engineer


#locations
#Ireland, Dublin, County Dublin, Ireland
#India, Hyderabad, India, Bangalore, India
#United States

jobs_pipeline = LinkedinJobsPipeline()
jobs_df = jobs_pipeline.run_jobs_extraction_pipeline(keywords="Software Developer",location= "Ireland",limit=50,isRefreshData=True,use_async=False)
display(jobs_df)

Scraped 19 jobs
Extracted features for 19 jobs
Table created successfully
Inserted the data into jobs successfully with 19 records


,title,job_link,company,seniority_level,employment_type,salary,job_function,industries,job_description,location,posted_date,programming_languages,technologies,education_requirements,key_skills,salary_range,salary_max,salary_min
0,"Frontend Software Engineer, Remediations User ...",https://ie.linkedin.com/jobs/view/frontend-sof...,Stripe,Mid-Senior level,Full-time,"€83,200 - €124,800.",Engineering and Information Technology,"Software Development, Financial Services, and ...",Who we areAbout StripeStripe is a financial in...,"Dublin, County Dublin, Ireland",None,"Angular, Css, Html, React, Vue",,[Engineering],"Angular, Css, Html, React, Vue","{'min': 83200, 'max': 124800}",124800.0,83200.0
1,Software Engineer,https://ie.linkedin.com/jobs/view/software-eng...,Udemy,Not Applicable,Full-time,N/A,Engineering and Information Technology,E-Learning Providers,Join Udemy. Help define the future of learning...,"Dublin, County Dublin, Ireland",None,"Css, Html, Java, Kotlin, Spring, Sql, Typescript","Ai, Graphql, Kafka, Kubernetes, Mysql, Redis","[Ms, Bs, Degree]","Ai, Css, Graphql, Html, Java, Kafka, Kotlin, K...",None,NaN,NaN
2,Software Engineer,https://ie.linkedin.com/jobs/view/software-eng...,Tines,Mid-Senior level,Full-time,N/A,Engineering,Software Development,Founded in 2018 with co-headquarters in Dublin...,Ireland,None,"React, Ruby, Typescript","Ai, Api, Aws, Docker, Redis",[Engineering],"Ai, Api, Aws, Docker, React, Redis, Ruby, Type...",None,NaN,NaN
3,"Intern, Software Engineer",https://ie.linkedin.com/jobs/view/intern-softw...,Autodesk,Entry level,Full-time,N/A,Engineering and Information Technology,"Software Development, Design Services, and IT ...",Job Requisition ID #25WD92004Position Overview...,"Dublin, County Dublin, Ireland",None,"Python, Typescript","Agile, Ai, Rest, Scrum","[Stem, Engineering, Computer Science, Degree]","Agile, Ai, Python, Rest, Scrum, Typescript",None,NaN,NaN
4,"Backend Engineer/API, Payments and Risk",https://ie.linkedin.com/jobs/view/backend-engi...,Stripe,Mid-Senior level,Full-time,"€83,200 - €156,000.",Engineering and Information Technology,"Software Development, Financial Services, and ...","Note: if you are an intern, new grad, or staff...","Dublin, County Dublin, Ireland",None,,"Api, Aws, Docker, Graphql, Kubernetes",[Engineering],"Api, Aws, Docker, Graphql, Kubernetes","{'min': 83200, 'max': 156000}",156000.0,83200.0
5,Software Engineer Graduate 2025/2026,https://ie.linkedin.com/jobs/view/software-eng...,Arista Networks,Mid-Senior level,Full-time,N/A,Engineering,Computer Networking,Company DescriptionArista Networks is an indus...,"Dublin, County Dublin, Ireland",None,"Go, Java, Node, Python, React","Git, Jenkins, Linux",[Engineering],"Git, Go, Java, Jenkins, Linux, Node, Python, R...",None,NaN,NaN
6,Backend Developer,https://ie.linkedin.com/jobs/view/backend-deve...,Vitalograph,Mid-Senior level,Full-time,N/A,Manufacturing,Medical Equipment Manufacturing,Job DescriptionVitalograph is the world’s lead...,"Limerick, County Limerick, Ireland",None,"Java, Node, Python, Sql","Api, Git, Graphql, Microservices","[Engineering, Computer Science, Degree]","Api, Git, Graphql, Java, Microservices, Node, ...",None,NaN,NaN
7,Software Engineer,https://ie.linkedin.com/jobs/view/software-eng...,Rithum,Entry level,Full-time,$50,Engineering and Information Technology,Software Development,Rithum™ is the world’s most trusted commerce n...,Ireland,None,"Angular, Java, Javascript, Node, Python, React...","Agile, Ai, Aws, Azure, Machine Learning, Pytor...","[Engineering, Computer Science, Degree]","Agile, Ai, Angular, Aws, Azure, Java, Javascri...","{'min': 50, 'max': 50}",50.0,50.0
8,Fullstack Developer - Fully Remote,https://ie.linkedin.com/jobs/view/fullstack-de...,Istituto Formativo Aladia,Not Applicable,Full-time,$5.00 – $20.00,Engineering and Information Technology,Education,About The RoleAbout Us:Aladia is a fast-growin...,Ireland,None,"Css, Html, Javascript, Node, Vue","Agile, Api, Ci/Cd, Docker, Git, Microserv

Implementation of FLASK API endpoints to extract jobs, applying various filters and database operation if required.

API Endpoints:
- Health & Status: GET http://localhost:5000/health API health check
- Job Extraction

  •	GET http://localhost:5000/jobs/extract?keywords=Python&location=Ireland&refresh_data=true

  •	Extracts fresh jobs from LinkedIn

  •	refresh_data=true - Replace all data

  •	refresh_data=false - Append new data (default)

- Job Retrieval

  •	GET http://localhost:5000/jobs - Get all jobs

  •	GET http://localhost:5000/jobs?company=Google - Filter by company

  •	GET http://localhost:5000/jobs?employment_type=Full-time - Filter by employment type

  •	GET http://localhost:5000/jobs?salary_min=100000&salary_max=200000 - Filter by salary

  •	GET http://localhost:5000/jobs?programming_language=Python - Filter by language

  •	GET http://localhost:5000/jobs?technology=Docker - Filter by technology

  •	GET http://localhost:5000/jobs/1 - Get specific job by ID

In [ ]:
from flask import Flask,request,jsonify
import pandas as pd
from typing import List,Dict

app = Flask(__name__)
jobs_pipeline = LinkedinJobsPipeline()
db_manager = DatabaseManager()

@app.route('/',methods=['GET'])
def home():
    return "Welcome to linkedin jobs scraper API, use 'jobs/extract' to extract the jobs and 'jobs/' to view all the jobs"

@app.route('/jobs/extract',methods=['GET'])
def extract_jobs():
    keywords = request.args.get('keywords')
    location = request.args.get('location')
    is_refresh_data = request.args.get('refresh_data',type=bool,default=False)

    if not keywords or not location:
        return jsonify({'message': 'To extract jobs please provide keywords and location as query parameter. Example: /jobs/extract?keywords=Software%20Developer&location=Dublin"'}), 400

    try:
        print(f"Extracting jobs for keywords : {keywords} and location: {location} and refresh data:{is_refresh_data}")
        extracted_df = jobs_pipeline.run_jobs_extraction_pipeline(keywords,location,isRefreshData=is_refresh_data)

        if extracted_df.empty:
            return jsonify({'message': 'No data extracted for the provided keyowrds and location'}), 404

        jobs_list = extracted_df.to_dict(orient='records')
        return jsonify({'message':f'Successfully extracted {len(jobs_list)} jobs','jobs_count':len(jobs_list),'jobs':jobs_list}), 200
    except Exception as e:
        return jsonify({'error': f'Error extracting jobs: {str(e)}'}), 500


@app.route('/jobs',methods=['GET'])
def get_jobs():
    try:
        db_manager._connect()
        jobs_df = db_manager.get_data()
        db_manager._close()

        if jobs_df.empty:
            return jsonify({'message': 'No jobs found in the database'}), 404

        #filtering as per query params
        company = requests.args.get('company')
        seniority_level = requests.args.get('seniority_level')
        employment_type = requests.args.get('employment_type')
        min_salary = requests.args.get('min_salary')
        max_salary = requests.args.get('max_salary')
        programming_language = requests.args.get('programming_language')
        technology = requests.args.get('technology')
        sort_by = requests.args.get('sort_by','posted_date')
        order_by= requests.args.get('order_by','desc')

        filtered_jobs=jobs_df.copy()

        if company:
            filtered_jobs=filtered_jobs[filtered_jobs['company'].str.contains(company,case=False,na=False)]
        if seniority_level:
            filtered_jobs=filtered_jobs[filtered_jobs['seniority_level'].str.contains(seniority_level,case=False, na=False)]
        if employment_type:
            filtered_jobs=filtered_jobs[filtered_jobs['employment_type'].str.contains(employment_type,case=False, na=False)]
        if min_salary:
            filtered_jobs=filtered_jobs[filtered_jobs['salary_min']>=int(min_salary)]
        if max_salary:
            filtered_jobs=filtered_jobs[filtered_jobs['salary_max']<=int(max_salary)]
        if programming_language:
            filtered_jobs=filtered_jobs[filtered_jobs['programming_languages'].str.contains(programming_language,case=False, na=False)]
        if technology:
            filtered_jobs=filtered_jobs[filtered_jobs['technologies'].str.contains(technology,case=False, na=False)]


        #sorting
        if sort_by in filtered_jobs.columns:
            ascending=True if order_by.lower()=='asc' else False

            if sort_by in ['salary_max','salary_min']:
                filtered_jobs[sort_by]=pd.numeric(filtered_jobs[sort_by], errors='coerce')

            filtered_jobs=filtered_jobs.sort_values(by=sort_by,ascending=ascending)

        jobs_list = filtered_jobs.to_dict(orient='records')
        return jsonify({'message':f'Successfully filtered the jobs','jobs_count':len(jobs_list),'jobs':jobs_list}), 200
    except Exception as e:
        return jsonify({'error': f'Error getting jobs: {str(e)}'}), 500

@app.route('/jobs/<int:job_id>',methods=['GET'])
def get_job_by_id(job_id):
    try:
        db_manager._connect()
        job_df = db_manager.cursor.execute(f'SELECT * FROM jobs WHERE id={job_id}').fetchall()
        columns=[description[0] for description in db_manager.cursor.description]
        db_manager._close()

        if jobs_df.empty:
            return jsonify({'message': f'No job found with id: {job_id}'}), 404

        job_dict=dict(zip(columns,jobs_df[0]))
        return jsonify({'message':f'Successfully fetched the job','job':job_dict}), 200
    except Exception as e:
        return jsonify({'error': f'Error getting job by id: {str(e)}'}), 500

# if __name__ == '__main__':
#     app.run(debug=True)

Adding Unit test cases to test all the classes

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import pytest

#scrapper test cases
class Test_Scrapper():

    def make_html_card(self,title='Test job',company='Google',location='Ireland',posted_date='2025-12-01'):
        return f"""
            <div class="base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card">
              <h3 class="base-search-card__title">{title}</h3>
              <h4 class="base-search-card__subtitle">{company}</h4>
              <span class="job-search-card__location">{location}</span>
              <time class="job-search-card__listdate">{posted_date}</time>
            </div>
          """
    def test_get_jobs_list_from_htmlcard(self):
        html_card=self.make_html_card()
        scraper=LinkedinJobScraper()
        soup=BeautifulSoup(html_card,'lxml')
        cards=soup.find_all('div', class_='base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card')
        jobs_list=scraper.get_jobs_list(cards,use_async=False)
        assert isinstance(jobs_list,list)
        assert len(jobs_list)==1
        assert jobs_list[0]['Job Title']=='Test job'

    def test_extract_salary_from_job_description(self):
        scraper=LinkedinJobScraper()
        html = '<div>Salary: $70k - $90k per year</div>'
        assert scraper.extract_salary_from_jobdescription(html) in ['$70k - $90k']

#feature extract test cases
def test_extract_programming_languages(self):
    fe=FeatureExtraction()
    text="Looking for .NET developer with experience in c# and react"
    languages=fe.extract_programming_languages(text)
    assert '.NET' in languages
    assert 'c#' in languages
    assert 'react' in  languages

def test_extract_technologies_and_skills(self):
    fe=FeatureExtraction()
    text="Experience with Docker, Azure and Git is required"
    technologies=fe.extract_technologies(text)
    skills=fe.extract_key_skills(text)
    assert 'Azure' in technologies
    assert 'Git' in technologies
    assert 'Docker' in skills

def test_extract_salary_ranges(self):
    fe=FeatureExtraction()
    job={'Salary':'$50k - $70k'}
    salary_range=fe.extract_salary_range(job)
    assert salary_range['min']==50000
    assert salary_range['max']==70000

#data transformer test cases
def test_clean_description(self):
    dt=DataTransformer()
    raw_text='<p>Hello/n/n/nPython</p>'
    cleaned_text=dt._clean_description(raw_text)
    assert 'Hello' in cleaned_text and 'Python' in cleaned_text

def test_transform_job_and_salary_extract(self):
    dt=DataTransformer()
    job={
        'title': 'Software Engineer',
        'company': 'Google',
        'location': 'Dublin',
        'salary': '$190k - $220k',
        'job_link':'https:/google.com',
        'seniority_level': 'SDE2',
        'employment_type': 'Full Time',
        'job_description':'<div>Test</div>',
        'posted_date':'2025-12-01',
        'programming_languages':['Python','Go'],
        'technologies':['GCP'],
        'key_skills': ['Python']
    }
    transformed_job=dt.transform_job(job)
    assert transformed_job['salary_max']==200000
    assert transformed_job['programming_languages']!=''

#db interaction test cases
def test_database_create_insert_get_reset(self):
    db_manager=DatabaseManager(db_name='jobs.db')
    db_manager.create_table(table_name='jobs')
    df = pd.DataFrame([{
            'title': 'Dev',
            'job_link': 'http://example.com',
            'company': 'Acme',
            'seniority_level': 'Mid',
            'employment_type': 'Full-time',
            'salary': '$50k',
            'job_function': 'Engineering',
            'industries': 'Tech',
            'job_description': 'Test',
            'location': 'Dublin',
            'posted_date': '2025-12-01',
            'programming_languages': 'Python',
            'technologies': 'Docker',
            'key_skills': 'Python',
            'salary_max': 50000,
            'salary_min': 50000
    }])

    db_manager.insert_data(df,table_name='jobs')
    data=db_manager.get_data(table_name='jobs')
    assert not data.empty

    db_manager.reset_database(table_name='jobs')
    data2=db_manager.get_data(table_name='jobs')
    assert data2.empty or isinstance(data2,pd.DataFrame)


#pipeline test cases
def test_pipeline_with_mocked_data(self,monkeypatch):
    df = pd.DataFrame([{
        'Job Title': 'Dev',
            'Job_Link': 'http://example.com',
            'Company': 'Acme',
            'Location': 'Dublin',
            'Posted Date': '2021-12-01',
            'Seniority Level': 'Mid',
            'Employment Type': 'Full-time',
            'Job Description': 'Looking for Python and Docker',
            'Salary': '$50k - $70k',
            'Job Function': 'Engineering',
            'Industries': 'Tech'
    }])

    pipeline=LinkedinJobsPipeline(db_name='jobs.db')
    monkeypatch.setattr(pipeline.scraper,'scrape_jobs',lambda *args, **kwargs:df)
    result_df=pipeline.run_jobs_extraction_pipeline('Python','Dublin', limit=25,isRefreshData=True)
    assert not result_df.empty
    assert 'Job Title' in result_df.columns or 'title' in result_df.columns